In [44]:
from __future__ import division
from pyomo.environ import *
import argparse
from pyomo.opt import SolverStatus, TerminationCondition
import pandas as pd
import pyutilib.services
import pickle
import random
import sqlite3
import copy
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

Results_folder = "./results/"

import numpy as np
import pandas as pd

INTEREST = 0.03
SCENARIOS = 1
SALV_PRICE = 0
    
SALVAGE = ""

OPT="_NPV"

OPT = OPT + "_"+str(SALV_PRICE)

#REQUIRES CBC TO BE INSTALLED ON THE MACHINE USED!!!  See https://github.com/coin-or/Cbc -- OTHER SOLVERS CAN BE USED -- 
#THEN CHANGE this line: "opt = SolverFactory('cbc') #Here we use the cbc solver -- open source software"

path = "/scratch/project_2003638/PLAN_REPLAN/"#ADJUST TO OWN PATH #"c:/mytemp/avohakkutpois/Files_for_optimization/temp/"


class optimization:
    def __init__(self):
        c = 0
        # Load data and preprocess by extracting branch and id from the id column
        data_opt =  pd.read_csv(path+"Data_Juliette.csv")
        data_opt['branch'] = [int(str(i)[-2:]) for i in data_opt['id']]
        data_opt['id'] = [int(str(i)[:-2]) for i in data_opt['id']]

        for k in range(0,SCENARIOS):
            if k == 0:
                dat = data_opt
                dat['iteration']=k
        data_opt = dat
        combinations = 1

        #CREATE replicates with varying iterations
        # Data structure initialization
        all_data = data_opt

        Index_values = all_data.set_index(['id','branch']).index.unique()
        all_data = all_data.set_index(['id','branch','year','iteration'])
        AREA = all_data.loc[slice(None),0,2016,all_data.index.get_level_values(3).min()]['AREA']
        all_data = all_data.fillna(0)
        all_data['year'] = all_data.index.get_level_values(2)
        self.data_opt=all_data

        self.combinations = 1

        #CREATE replicates with varying iterations

        self.all_data = self.data_opt
        self.Index_values = self.all_data.drop(['year'], axis=1).reset_index().set_index(['id','branch']).index.unique()#all_data.set_index(['id','branch']).index.unique()
        self.AREA = self.all_data.loc[slice(None),0,2016,self.all_data.index.get_level_values(3).min()]['AREA']
        self.all_data = self.all_data.fillna(0)

        self.createModel()

    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        # Define sets, variables, and constraints for the optimization problem
        
        self.model1 = ConcreteModel()

        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0))))
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(2))))
        self.model1.iteration = Set(initialize = list(set(self.all_data.index.get_level_values(3))))        
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.scen_index = Set(initialize= [i for i in range(0,self.combinations)])
        self.model1.Index_values = self.Index_values

        # Indexes (stand, regime)-- excludes those combinations that have no regimes simulated

        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: #stand_set
                index.append((s,r))
            return index
        self.model1.index1 = Set(dimen=2, initialize=index_rule)

        self.model1.X1 = Var(self.model1.index1, within=NonNegativeReals, bounds=(0,1), initialize=1)

        self.all_data['year'] = self.all_data.index.get_level_values(2)

        #objective function:
        def outcome_rule(model1):
            return sum((self.all_data.Harvested_V.loc[(s,r,k,it)]*self.all_data.AREA.loc[(s,r,k,it)]* self.model1.X1[(s,r)])/((1+INTEREST)**(2.5+self.all_data.year[(s,r,k,it)]))  for (s,r) in self.model1.index1 for k in self.model1.year for it in self.model1.iteration)
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)

    def solve(self):
        # Specify the solver and solve the model
        opt = SolverFactory('cbc') #Here we use the cbc solver -- open source software
        self.results = opt.solve(self.model1,tee=False) #We solve a problem, but do not show the solver output

# Create an optimization object        
t1 = optimization()
t2 = copy.deepcopy(t1)

# Modify the optimization model to focus on. Currently maximizing the NPV.

### MAX MINIMUM NPV:
#Max min
try:
    t2.model1.del_component(t2.model1.NPV_INV)
except:
    print("NONE")

#A function that evaluates NPV -- this includes wind disturbed salvage (however, the data doesn't include -- so it could be simplified.)    
t2.model1.NPV= Var(within=NonNegativeReals)
def NPV_INVENTORY(model1,it):
    row_sum = sum(((t2.all_data.income.loc[(s,r,k,it)]+t2.all_data.natural_rm_wind.loc[(s,r,k,it)]*SALV_PRICE)*t2.all_data.AREA.loc[(s,r,k,it)]* t2.model1.X1[(s,r)])/((1+INTEREST)**(2.5+t2.all_data.year[(s,r,k,it)]-2016))  for (s,r) in t2.model1.index1 for k in t2.model1.year)+sum((t2.all_data.PV.loc[(s,r,max(t2.all_data.year[(s,r,slice(None),t2.all_data.index.get_level_values(3).min())]),it)]*t2.all_data.AREA.loc[(s,r,max(t2.all_data.year[(s,r,slice(None),t2.all_data.index.get_level_values(3).min())]),it)]* t2.model1.X1[(s,r)])/((1+INTEREST)**(2.5+max(t2.all_data.year[(s,r,slice(None),t2.all_data.index.get_level_values(3).min())])-2016))  for (s,r) in t2.model1.index1)
    return t2.model1.NPV<=row_sum
t2.model1.NPV_INV= Constraint(t2.model1.iteration,rule=NPV_INVENTORY)

#Likely not needed -- used if constraints for even flow are present, and tries to remove them.
try:
    t2.model1.del_component(t2.model1.EVEN_inc)
    t2.model1.del_component(t2.model1.EF)
except:
    print("NONE")

#Objective function -- maximizing NPV    
def outcome_rule(model1):
    return t2.model1.NPV
t2.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

# Solve the modified optimization model
t2.solve()

#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t2.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t2.model1.X1[(s,r)].value]
        data = {"id":st,"branch":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['id','branch'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t2.all_data, left_index=True, right_index=True, how='left')

# Save results to a CSV file
merged_df.to_csv(path+"output.csv")

NONE
NONE
    'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This
    is usually indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().


In [47]:
#merged_df = pd.merge(dec, t2.all_data, on=['id', 'branch'], how='left')
merged_df = dec.merge(t2.all_data, left_index=True, right_index=True, how='left')

(merged_df['value']*merged_df['AREA']).sum()/6
#merged_df

287.4630006998778

In [43]:
#data_opt =  pd.read_csv("/scratch/project_2003638/PLAN_REPLAN/WindRisk.csv")
data_opt[data_opt['iteration']==162].drop('iteration',axis=1).to_csv("/scratch/project_2003638/PLAN_REPLAN/Data_Juliette.csv")
#data_opt

In [1]:
from __future__ import division
from pyomo.environ import *
import argparse
from pyomo.opt import SolverStatus, TerminationCondition
import pandas as pd
import pyutilib.services
import pickle
import random
import sqlite3
import copy
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

Results_folder = "./results/"

import numpy as np
import pandas as pd

INTEREST = 0.03
SCENARIOS = 1
SALV_PRICE = 0#args.SP
    
SALVAGE = ""
#if args.OPT == "NPV":
OPT="_NPV"
#else:
#    OPT = "_CVAR"
OPT = OPT + "_"+str(SALV_PRICE)

#REQUIRES CBC TO BE INSTALLED ON THE MACHINE USED!!!  See https://github.com/coin-or/Cbc -- OTHER SOLVERS CAN BE USED -- 
#THEN CHANGE this line: "opt = SolverFactory('cbc') #Here we use the cbc solver -- open source software"

path = "/scratch/project_2003638/PLAN_REPLAN/DB/"#ADJUST TO OWN PATH #"c:/mytemp/avohakkutpois/Files_for_optimization/temp/"
path_cluster = "./Data/"

In [13]:
c = 0

data_opt =  pd.read_csv("/scratch/project_2003638/PLAN_REPLAN/WindRisk.csv")
data_opt['branch'] = [int(str(i)[-2:]) for i in data_opt['id']]
data_opt['id'] = [int(str(i)[:-2]) for i in data_opt['id']]

#CLUSTER_1 = pd.read_csv(path_cluster+"cluster_1.csv",header=None)
#CLUSTER_2 = pd.read_csv(path_cluster+"cluster_2.csv",header=None)
#CLUSTER_3 = pd.read_csv(path_cluster+"cluster_3.csv",header=None)

for k in range(0,SCENARIOS):
    if k == 0:
        dat = data_opt[(data_opt["iteration"] == 0) ]
        dat['iteration']=k
    else:
        dat1 = data_opt[(data_opt['id'].isin(list(CLUSTER_1[0]))) & (data_opt["iteration"] == random.choice(VV[0])) | (data_opt['id'].isin(list(CLUSTER_2[0]))) & (data_opt["iteration"] == random.choice(VV[0]))| (data_opt['id'].isin(list(CLUSTER_3[0]))) & (data_opt["iteration"] == random.choice(VV[0]))]
        dat1['iteration']=k
        dat =pd.concat([dat,dat1])
data_opt2 = dat
combinations = 1

#CREATE replicates with varying iterations

all_data = data_opt2

Index_values = all_data.set_index(['id','branch']).index.unique()
all_data = all_data.set_index(['id','branch','year','iteration'])
AREA = all_data.loc[slice(None),0,2016,all_data.index.get_level_values(3).min()]['AREA']
all_data = all_data.fillna(0)
all_data['year'] = all_data.index.get_level_values(2)
data_opt2=all_data

combinations = 1


In [14]:
data_opt

,id,branch,year,iteration,V,income,Harvested_V,Harvested_V_log,Harvested_V_pulp,AREA,...,COWBERRY,CARBON_SOIL_Update,BM_total,DEAD_WOOD_DIVERSITY,scenic,MAIN_SP,H_dom,THIN,SC,natural_rm_wind
0,1000,0,2016.0,0,115.633316,0.000000,0.00000,0.000000,0.000000,0.358,...,11.829735,81729.984375,91436.968750,0.0956,4.278676,4.0,14.103016,0,3.0,0.00000
1,1000,0,2021.0,0,147.821259,0.000000,0.00000,0.000000,0.000000,0.358,...,11.224109,82959.656250,113110.695312,0.2145,4.470498,4.0,15.340117,0,3.0,0.00000
2,1000,0,2026.0,0,5.249289,0.000000,0.00000,0.000000,0.000000,0.358,...,34.184933,77559.242188,2789.883057,0.2325,5.570047,4.0,19.892017,0,3.0,169.93367
3,1000,0,2031.0,0,4.413198,0.000000,0.00000,0.000000,0.000000,0.358,...,35.105885,70595.210938,3487.127197,0.3274,5.915336,4.0,22.734344,0,3.0,0.00000
4,1000,0,2036.0,0,6.478335,0.000000,0.00000,0.000000,0.000000,0.358,...,32.330486,65639.234375,5117.913086,0.5190,5.781097,4.0,23.388220,0,3.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252155,1285,5,2021.0,203,37.731598,0.000000,0.00000,0.000000,0.000000,0.863,...,19.686104,78238.906250,31944.134766,1.0843,6.766620,4.0,19.960556,0,3.0,0.00000
2252156,1285,5,2026.0,203,32.875336,0.000000,0.00000,0.000000,0.000000,0.863,...,20.668291,75943.414062,28147.369141,1.3014,6.889361,4.0,19.985298,0,3.0,0.00000
2252157,1285,5,2031.0,203,29.460621,0.000000,0.00000,0.000000,0.000000,0.863,...,21.435759,73180.523438,25570.281250,0.9457,7.016605,4.0,20.005831,0,3.0,0.00000
2252158,1285,5,2036.0,203,0.000000,949.295164,23.89559,15.021458,8.874132,0.863,...,22.031942,70291.328125,23761.728516,0.5962,4.471000,4.0,0.000000,0,3.0,0.00000


In [15]:
all_data

V      income  Harvested_V  \
id   branch year   iteration                                        
1000 0      2016.0 0          115.633316    0.000000      0.00000   
            2021.0 0          147.821259    0.000000      0.00000   
            2026.0 0            5.249289    0.000000      0.00000   
            2031.0 0            4.413198    0.000000      0.00000   
            2036.0 0            6.478335    0.000000      0.00000   
...                                  ...         ...          ...   
1285 5      2021.0 0           37.731598    0.000000      0.00000   
            2026.0 0           32.875336    0.000000      0.00000   
            2031.0 0           29.460621    0.000000      0.00000   
            2036.0 0            0.000000  949.295164     23.89559   
            2041.0 0            0.000000    0.000000      0.00000   

                              Harvested_V_log  Harvested_V_pulp   AREA  \
id   branch year   iteration                                             
1000 0      2016.0 0                 0.000000          0.000000  0.358   
            2021.0 0                 0.000000          0.000000  0.358   
            2026.0 0                 0.000000          0.000000  0.358   
            2031.0 0                 0.000000          0.000000  0.358   
            2036.0 0                 0.000000          0.000000  0.358   
...                                       ...               ...    ...   
1285 5      2021.0 0                 0.000000          0.000000  0.863   
            2026.0 0                 0.000000          0.000000  0.863   
            2031.0 0                 0.000000          0.000000  0.863   
            2036.0 0                15.021458          8.874132  0.863   
            2041.0 0                 0.000000          0.000000  0.863   

                                       PV  HSI_RL_S1  HSI_RL_S2  HSI_RL_S3  \
id   branch year   iteration                                                 
1000 0      2016.0 0          3682.871582        0.0   0.109129   0.000000   
            2021.0 0          4790.845215        0.0   0.150987   0.090865   
            2026.0 0          3937.197510        0.0   0.145295   0.030152   
            2031.0 0          4096.666016        0.0   0.135419   0.023357   
            2036.0 0           761.690002        0.0   0.134318   0.024885   
...                                   ...        ...        ...        ...   
1285 5      2021.0 0          4155.308594        0.0   0.246248   0.068737   
            2026.0 0          4768.164062        0.0   0.214056   0.072566   
            2031.0 0          5287.372070        0.0   0.186310   0.058170   
            2036.0 0           272.252716        0.0   0.170224   0.071543   
            2041.0 0          1938.397827        0.0   0.170224   0.071543   

                              ...  CARBON_SOIL_Update       BM_total  \
id   branch year   iteration  ...                                      
1000 0      2016.0 0          ...        81729.984375   91436.968750   
            2021.0 0          ...        82959.656250  113110.695312   
            2026.0 0          ...        77559.242188    2789.883057   
            2031.0 0          ...        70595.210938    3487.127197   
            2036.0 0          ...        65639.234375    5117.913086   
...                           ...                 ...            ...   
1285 5      2021.0 0          ...        78238.906250   31944.134766   
            2026.0 0          ...        75943.414062   28147.369141   
            2031.0 0          ...        73180.523438   25570.281250   
            2036.0 0          ...        70291.328125   23761.728516   
            2041.0 0          ...        64765.839844       0.000000   

                              DEAD_WOOD_DIVERSITY    scenic  MAIN_SP  \
id   branch year   iteration                                           
1000 0      2016.0 0                       0.0956  4.278676      4.0   
          

In [10]:
data_optf =  pd.read_csv("/scratch/project_2003638/PLAN_REPLAN/WindRisk.csv")

In [11]:
data_optf

,id,branch,year,iteration,V,income,Harvested_V,Harvested_V_log,Harvested_V_pulp,AREA,...,COWBERRY,CARBON_SOIL_Update,BM_total,DEAD_WOOD_DIVERSITY,scenic,MAIN_SP,H_dom,THIN,SC,natural_rm_wind
0,100000,0,2016.0,0,115.633316,0.000000,0.00000,0.000000,0.000000,0.358,...,11.829735,81729.984375,91436.968750,0.0956,4.278676,4.0,14.103016,0,3.0,0.00000
1,100000,0,2021.0,0,147.821259,0.000000,0.00000,0.000000,0.000000,0.358,...,11.224109,82959.656250,113110.695312,0.2145,4.470498,4.0,15.340117,0,3.0,0.00000
2,100000,0,2026.0,0,5.249289,0.000000,0.00000,0.000000,0.000000,0.358,...,34.184933,77559.242188,2789.883057,0.2325,5.570047,4.0,19.892017,0,3.0,169.93367
3,100000,0,2031.0,0,4.413198,0.000000,0.00000,0.000000,0.000000,0.358,...,35.105885,70595.210938,3487.127197,0.3274,5.915336,4.0,22.734344,0,3.0,0.00000
4,100000,0,2036.0,0,6.478335,0.000000,0.00000,0.000000,0.000000,0.358,...,32.330486,65639.234375,5117.913086,0.5190,5.781097,4.0,23.388220,0,3.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252155,128505,0,2021.0,203,37.731598,0.000000,0.00000,0.000000,0.000000,0.863,...,19.686104,78238.906250,31944.134766,1.0843,6.766620,4.0,19.960556,0,3.0,0.00000
2252156,128505,0,2026.0,203,32.875336,0.000000,0.00000,0.000000,0.000000,0.863,...,20.668291,75943.414062,28147.369141,1.3014,6.889361,4.0,19.985298,0,3.0,0.00000
2252157,128505,0,2031.0,203,29.460621,0.000000,0.00000,0.000000,0.000000,0.863,...,21.435759,73180.523438,25570.281250,0.9457,7.016605,4.0,20.005831,0,3.0,0.00000
2252158,128505,0,2036.0,203,0.000000,949.295164,23.89559,15.021458,8.874132,0.863,...,22.031942,70291.328125,23761.728516,0.5962,4.471000,4.0,0.000000,0,3.0,0.00000
